# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Описание данных

**Признаки**
* `DateCrawled` — дата скачивания анкеты из базы
* `VehicleType` — тип автомобильного кузова
* `RegistrationYear` — год регистрации автомобиля
* `Gearbox` — тип коробки передач
* `Power` — мощность (л. с.)
* `Model` — модель автомобиля
* `Kilometer` — пробег (км)
* `RegistrationMonth` — месяц регистрации автомобиля
* `FuelType` — тип топлива
* `Brand` — марка автомобиля
* `Repaired` — была машина в ремонте или нет
* `DateCreated` — дата создания анкеты
* `NumberOfPictures` — количество фотографий автомобиля
* `PostalCode` — почтовый индекс владельца анкеты (пользователя)
* `LastSeen` — дата последней активности пользователя

**Целевой признак**
* Price — цена (евро)

## Подготовка данных

### Установка библиотек и настройка окружения

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
import shap

from IPython.display import display

from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from catboost import CatBoostClassifier

import scipy
import scipy.stats as stats
from scipy.stats import poisson

import phik
from phik import resources, report

warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format

### Инициализация полезных функций

In [ ]:
class DatasetSizeTracker:
    def __init__(self):
        self.initial_size = None
        self.previous_size = None

    def track(self, data):
        # Если исходный размер не был установлен, запоминаем его и текущий размер как первоначальный
        if self.initial_size is None:
            self.initial_size = len(data)
            self.previous_size = len(data)
            loss_from_initial = 0
            loss_from_previous = 0
            loss_percent_overoll = 0
        else:
            # Считаем, сколько строк потеряно относительно исходного размера и с прошлого вызова
            current_size = len(data)
            loss_from_initial = self.initial_size - current_size
            loss_from_previous = self.previous_size - current_size
            loss_percent_overoll = ((self.initial_size - current_size)/self.initial_size)*100
            # Обновляем предыдущий размер для следующего вызова
            self.previous_size = current_size
        
        return {"Потери от исходного размера": loss_from_initial, "Потери с предыдущего раза": loss_from_previous,
                'Общие потери в процентах' : loss_percent_overoll}

# Создание экземпляра класса для трекинга размера датасета
tracker = DatasetSizeTracker()

In [ ]:
# функция информации по таблице
def dataframe_summary(df, string):
    # Вывод общей информации
    print("Общая информация по таблице:", string)
    df.info()

    # Вывод статистического описания
    print("\n Статистическое описание:")
    display(df.describe().transpose())

    # Вывод случайных примеров
    print("\nСлучайные примеры:")
    display(df.sample(5))

    # Вывод количества строк и столбцов
    print("\nКоличество строк и столбцов:", df.shape)
    
    # Вывод количества явных дубликатов
    print("\nКоличество явных дубликатов:", df.duplicated().sum())
    print('')

In [ ]:
# функция поиска пропусков
def analyze_missing_values(df, string):
        total = df.isnull().sum().sort_values(ascending=False)
        percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
        missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        missing_data = missing_data.query('Total > 0')

        # Вывод информации о пропусках
        print(f"Пропуски в датафрейме {string}:\n{missing_data}\n")

        # Создание и отображение тепловой карты
        plt.figure(figsize=(12, 8))
        sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
        plt.title(f"Heatmap пропусков для {string}")
        plt.show()

In [ ]:
# Функция для преобразования названий в змеиный регистр (snake_case) 
def to_snake_case(name): 
    s1 = name[0].lower() 
    for c in name[1:]: 
        if c.isupper(): 
            s1 += '_' 
            s1 += c.lower() 
        else: 
            s1 += c 
    return s1 
 

In [ ]:
def fill_missing_values_by_brand_and_model(data, column_name):
    """
    Заполняет пропуски в указанной колонке на основе самого частого значения,
    характерного для сгруппированных данных по 'brand' и 'model'.
    
    Parameters:
    data (pd.DataFrame): Исходный датафрейм.
    column_name (str): Название колонки, в которой нужно заполнить пропуски.
    
    Returns:
    pd.DataFrame: Датафрейм с заполненными пропусками в указанной колонке.
    """
    # Считаем самое частое значение для каждой группы 'brand' и 'model'
    most_frequent_values = data.groupby(['brand', 'model'])[column_name].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    
    # Функция для заполнения пропуска
    def fill_value(row):
        if pd.isnull(row[column_name]):
            return most_frequent_values.get((row['brand'], row['model']), row[column_name])
        else:
            return row[column_name]
    
    # Заполнение пропусков
    data[column_name] = data.apply(fill_value, axis=1)
    
    return data

### Обзор данных, работа с дубликатами, пропусками и выбросами

План работы:
1) Анализ описательной статитстики по данным;
2) Очистка явных дубликатов;
3) Обработка пропусков;
4) Обработка выбросов;
5) Анализ визуальной статистики.

#### Обзор данных

In [ ]:
data = pd.read_csv('autos.csv')

# Создадим отдельный датасет, который мы оставим в исходном состоянии. 
# Все необходимые преобразования
# data_model = data.copy()

In [ ]:
from sklearn.model_selection import train_test_split

def get_stratified_sample(data, stratify_column='column_for_stratification', sample_size=100000):
    """
    Возвращает случайные sample_size строк из датафрейма data, сохраняя пропорциональное представление значений в stratify_column.
    
    Parameters:
    data (pd.DataFrame): Исходный датафрейм.
    stratify_column (str): Название колонки, по которой нужно стратифицировать выборку.
    sample_size (int): Размер выборки.
    
    Returns:
    pd.DataFrame: Датафрейм, содержащий случайную стратифицированную выборку указанного размера.
    """
    # Разделение данных на "обучающий" набор (который будет отброшен) и "тестовый" набор, который является нужной выборкой
    _, data_sample = train_test_split(data, test_size=sample_size, stratify=data[stratify_column], random_state=42)
    
    return data_sample

# Для использования функции, укажите DataFrame, колонку для стратификации и желаемый размер выборки:
sample_data = get_stratified_sample(data, 'Brand', 100000)

In [ ]:
dataframe_summary(data, 'Автомобили (data)')

In [ ]:
tracker = DatasetSizeTracker()

tracker.track(data)  # Посмотреть, сколько строк потеряно с исходного момента и с последнего вызова

Изменим названия переменных, избавимся от дубликатов

In [ ]:
data.columns = [to_snake_case(name) for name in data.columns] 
data.columns

In [ ]:
# удалим явные дубликаты
data.drop_duplicates(inplace=True)

В данных есть объявления с ценой равной 0. Тк это целевой признак - менять нельзя, но лучше удалить строки с такой ценой.

In [ ]:
print('Сколько людей готовы "подарить" автомобиль:',len(data.loc[data['price'] == 0]))
data = data.loc[data['price'] != 0]
print('Сколько людей готовы "подарить" автомобиль теперь:',len(data.loc[data['price'] == 0]))

In [ ]:
tracker.track(data) 

In [ ]:
analyze_missing_values(data, 'data')

Избавимся от явных дубликатов, возможно, после обработки датасета и удаления ненужных переменных дубликатов станет больше.

In [ ]:
data = data.drop(['date_crawled', 'registration_month', 'date_created', 
                  'number_of_pictures', 'postal_code', 'last_seen'], axis =1) 

In [ ]:
data.columns

In [ ]:
display(data[['brand', 'model','vehicle_type', 'fuel_type', 'gearbox', 'power' ,'repaired','price']].sample(10))

#### Работа с пропусками

##### Пропуски в `model`

Пропуски в `model`, на мой взгляд, критичны. Фактор модели может играть значительную роль в определении цены машины. Восполнить пропуски здесь никак, заглушка тоже может в данном случае не подойти или нарушить связи. 

In [ ]:
data = data.loc[~data['model'].isna()]
tracker.track(data) 

##### Пропуски в `repaired`

Пропусков в repaired довольно много, порядка 20% от изначального размера данных.\
Фактор довольно важный, при написании объявления обязательно указывать, если машина ремонтировалась. Возможно, значения в этом столбце пропущены, если машина не чинилась и продавцы оставляли незаполненным этот пропуск. \
Если качество модели будет неудовлетворительным, можно попробовать применить заглушку.


In [ ]:
data.repaired.value_counts()

In [ ]:
data.repaired.fillna('no', inplace=True)

In [ ]:
analyze_missing_values(data, 'data')

Остались пропуски в `vehicle_type`,  `fuel_type` и `gearbox` - их заполним на основе данных из `model` и `brand`. 

In [ ]:
print(data['vehicle_type'].value_counts())
print(data['fuel_type'].value_counts())
print(data['gearbox'].value_counts())

In [ ]:
# используем функцию для заполнения пропусков на основе сгрупиированных данных по brand и model
missing_list = ['vehicle_type', 'fuel_type', 'gearbox']
for i in missing_list:
    data = fill_missing_values_by_brand_and_model(data, i)


In [ ]:
print(data['vehicle_type'].value_counts())
print(data['fuel_type'].value_counts())
print(data['gearbox'].value_counts())

#### Обработка аномальных значений

In [ ]:
data.describe()

## Обучение моделей

## Анализ моделей

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей